# 一 函数准备

## 1.1 载入 Python 库

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import os, json
import cv2
from PIL import Image
from glob import glob

## 1.2 载入自写的函数

In [2]:
def get_scaler(img_arr,img_height,img_width):  
    """
    获得图片中刻度尺长度方向的像素数，对应物理长度 500 微米
    img_arr：矩阵格式的图片
    img_height：图片的高，即竖直方向的像素数
    img_width：图片的宽，即水平方向的像素数
    """  
    rd = img_arr[img_height//2:, img_width//2:, :] # 取图片的 1/4，右下那一块，right_down
    mask = (rd[..., 0] >= 240) * (rd[..., 1] <= 20) * (rd[..., 2] <= 20) # 红色像素为 True/1，非红像素为 False/0
    scaler = np.max(np.sum(mask, axis = 1)) # 计算水平方向上的红色像素数量，取最大值，即为刻度尺的像素数
    return(scaler) 

In [3]:
def get_center(cnts):
    """
    获得性状的中心点和半径
    cnts：轮廓点
    """  
    (x,y),radius = cv2.minEnclosingCircle(cnts)
    center = (int(x),int(y))
    radius = int(radius)
    return center,radius

In [4]:
def vb_initial_process(points):   
    """
    获得维管束性状的初级数据，包括单个维管束面积和单个维管束中心
    points：单个维管束的轮廓点
    """  
    if shape_type == "polygon":
        # 求面积
        area_pixel = cv2.contourArea(points)
        # 求中心
        vb_center = get_center(points)[0]
    elif shape_type == "circle":
        # 求面积
        vb_center, vb_round = points[0], points[1] # 求中心 
        r = math.dist(vb_center,vb_round)
        area_pixel = round(math.pi * r ** 2,1)
        
    return area_pixel,vb_center

In [5]:
def vb_more_process(area_pixel_union,vb_center_union,stem_center):  
    """
    进一步获得维管束性状数据，包括维管束数量、维管束像素面积之和、维管束平均像素面积
    以及维管束到茎中心的像素距离集、维管束到茎中心的平均像素距离
    area_pixel_union：维管束像素面积集
    vb_center_union：维管束中心集
    stem_center：茎中心 
    """  
    num = len(area_pixel_union) # 维管束数量
    area_sum_pixel = np.sum(area_pixel_union) # 维管束像素面积之和
    area_mean_pixel = np.mean(area_pixel_union) # 维管束平均像素面积
    center_dist_pixel_union = [math.dist(x,stem_center) for x in vb_center_union] # 维管束到茎中心的像素距离集
    center_dist_mean_pixel = np.mean(center_dist_pixel_union) # 维管束到茎中心的平均像素距离

    return num,area_sum_pixel,area_mean_pixel,center_dist_pixel_union,center_dist_mean_pixel  

In [6]:
def write2excel(df,savepath,sheet_name):
    """
    将数据写入已存在的 excel 表
    df：DataFrame
    savepath：excel 所在的路径
    sheet_name：表格名
    """
    import openpyxl

    wb = openpyxl.load_workbook(savepath)
    #如果有多个模块可以读写excel文件，这里要指定engine，否则可能会报错
    writer = pd.ExcelWriter(savepath,engine='openpyxl')
    #没有下面这个语句的话excel表将完全被覆盖
    writer.book = wb
 
    #如果有相同名字的工作表，新添加的将命名为Sheet21，如果Sheet21也有了就命名为Sheet22，不会覆盖原来的工作表
    df.to_excel(writer,sheet_name = sheet_name)
    writer.save()
    writer.close()

# 二 数据处理

In [8]:
wd = "/Users/sherlock/Documents/bioinformatics/维管束/" # 设定工作路径
# wd = "your/working/directory/"
json_path = wd + "维管束切片图像/0620删掉多余比例尺/*.json" # 文件路径格式
jsons = glob(json_path) # 查找所有符合格式的文件
jsons[:5] # 查看前五条文件路径

['/Users/sherlock/Documents/bioinformatics/维管束/维管束切片图像/0620删掉多余比例尺/L47.json',
 '/Users/sherlock/Documents/bioinformatics/维管束/维管束切片图像/0620删掉多余比例尺/L50.json',
 '/Users/sherlock/Documents/bioinformatics/维管束/维管束切片图像/0620删掉多余比例尺/F11.json',
 '/Users/sherlock/Documents/bioinformatics/维管束/维管束切片图像/0620删掉多余比例尺/F12.json']

## 2.1 生成直接导出的数据

In [9]:
# 导出性状数据
all_res = {} # 创建字典以存储结果

for json_file in jsons:
    sample_res = {} # 创建字典存储每一个样本的数据
    filename = os.path.splitext(json_file)[0] 
    sample_name = filename.split("/")[-2] + "/" + filename.split("/")[-1] # 获得样本名
    print(sample_name)
    
    # 第一步，处理样本图片文件，获得刻度尺长度方向上的像素数量  
    tif_file = filename + ".tif"
    bmp_file = filename + ".bmp"
    jpg_file = filename + ".jpg"
    # 导入图片
    if os.path.exists(tif_file):
        img = Image.open(tif_file)
    elif os.path.exists(bmp_file):
        img = Image.open(bmp_file)
    elif os.path.exists(jpg_file):
        img = Image.open(jpg_file)
    # 获得图片竖直方向和水平方向的像素数，以及图片矩阵
    img_h = img.height
    img_w = img.width
    img_arr = np.array(img)
    # 获得刻度尺长度方向上的像素数量 
    scaler = get_scaler(img_arr,img_h,img_w)
    print("比例尺像素长度：",scaler)
    sample_res["比例尺像素长度"] = scaler
    
    # 第二步，处理样本 json 文件，获得性状数据
    with open(json_file) as f:
        data = json.loads(f.read()) # 读入 json 文件    
        
        # 创建列表以存放性状数据集
        big_area_pixel_union, small_area_pixel_union = [], []
        big_center_union, small_center_union = [],[]
        hole_area_pixel_union = []
         
        for shape in data['shapes']:
            label = shape['label']
            shape_type = shape['shape_type']
            points = np.array(shape['points']).astype(int)
            
            # 2.1 数据提取
            # 2.1.1 空腔相关的数据提取，包括空腔像素周长、空腔像素面积
            if label == "hole":
                # 求面积
                hole_area_pixel = cv2.contourArea(points)
                print("hole_area_pixel:",hole_area_pixel)
                hole_area_pixel_union.append(hole_area_pixel)           
            
            # 2.1.2 茎相关的数据提取，包括茎中心、茎像素半径、茎像素周长、茎像素面积
            if label == "out":
                if shape_type == "polygon":
                    stem_center,steam_radius_pixel = get_center(points) # 求圆心和半径
                    print("stem_center:",stem_center)
                    print("*****")
#                     print("steam_radius_pixel:",steam_radius_pixel)                  
                    stem_perimeter_pixel = cv2.arcLength(points, True) # 求周长
#                     print("stem_perimeter_pixel:",stem_perimeter_pixel)                   
                    stem_area_pixel = cv2.contourArea(points)  # 求面积
#                     print("stem_area_pixel:",stem_area_pixel)
                                        
            # 2.1.3 维管束相关的数据提取  
            # 对大维管束
            if label == "wgs":
                area_pixel, vb_center = vb_initial_process(points)
                big_area_pixel_union.append(area_pixel)
                big_center_union.append(vb_center)
                     
            # 对小维管束
            if label == "small wgs" or label == "smwgs":
                area_pixel, vb_center = vb_initial_process(points)
                small_area_pixel_union.append(area_pixel)
                small_center_union.append(vb_center)
                
        
        # 2.2 数据赋值
        # 2.2.1 一级数据赋值
        sample_res["空腔像素面积集"] = hole_area_pixel_union   
        sample_res["空腔像素面积"] = np.sum(hole_area_pixel_union)
#             print("空腔像素面积集", sample_res["空腔像素面积集"])
#             print("空腔像素面积", sample_res["空腔像素面积"])

        sample_res["茎中心"] = stem_center  
        sample_res["茎像素半径"] = steam_radius_pixel
        sample_res["茎像素直径"] = steam_radius_pixel * 2
        sample_res["茎像素周长"] = stem_perimeter_pixel
        sample_res["茎像素面积"] = stem_area_pixel
        sample_res["茎像素面积_减空腔后"] = sample_res["茎像素面积"] - sample_res["空腔像素面积"]

        sample_res["大维管束像素面积集"] = big_area_pixel_union
        sample_res["大维管束中心集"] = big_center_union
#             print("big_area_pixel_union:",big_area_pixel_union)

        sample_res["小维管束像素面积集"] = small_area_pixel_union
        sample_res["小维管束中心集"] = small_center_union
#             print("small_area_pixel_union:",small_area_pixel_union)    
    
        # 进一步得到维管束性状
        vb_more_res = vb_more_process(big_area_pixel_union,big_center_union,stem_center)
        sample_res["大维管束数量"]  = vb_more_res[0]
        sample_res["大维管束像素面积之和"]  = vb_more_res[1] 
        sample_res["大维管束平均像素面积"] = vb_more_res[2]
        sample_res["大维管束到茎中心的像素距离集"] = vb_more_res[3]
        sample_res["大维管束到茎中心的平均像素距离"] = vb_more_res[4]
        
        vb_more_res = vb_more_process(small_area_pixel_union,small_center_union,stem_center)
        sample_res["小维管束数量"]  = vb_more_res[0]
        sample_res["小维管束像素面积之和"]  = vb_more_res[1] 
        sample_res["小维管束平均像素面积"] = vb_more_res[2]
        sample_res["小维管束到茎中心的像素距离集"] = vb_more_res[3]
        sample_res["小维管束到茎中心的平均像素距离"] = vb_more_res[4]
        
        # 2.2.2 二级数据赋值
        sample_res["总维管束数量"] = sample_res["大维管束数量"] + sample_res["小维管束数量"]
        sample_res["大维管束数量/总维管束数量（%）"] = 100 * sample_res["大维管束数量"] / sample_res["总维管束数量"]
        sample_res["总维管束像素面积之和"] = sample_res["大维管束像素面积之和"] + sample_res["小维管束像素面积之和"]
        sample_res["大维管束面积之和/总维管束面积之和（%）"] = 100 * sample_res["大维管束像素面积之和"] / sample_res["总维管束像素面积之和"]
        sample_res["大维管束面积之和/茎面积_减空腔后（%）"] = 100 * sample_res["大维管束像素面积之和"] / sample_res["茎像素面积_减空腔后"]
        sample_res["小维管束面积之和/茎面积_减空腔后（%）"] = 100 * sample_res["小维管束像素面积之和"] / sample_res["茎像素面积_减空腔后"]
        sample_res["大维管束到茎中心的平均距离/茎半径（%）"] = 100 * sample_res["大维管束到茎中心的平均像素距离"] / sample_res["茎像素半径"]
        sample_res["小维管束到茎中心的平均距离/茎半径（%）"] = 100 * sample_res["小维管束到茎中心的平均像素距离"] / sample_res["茎像素半径"]
        sample_res["皮像素厚度"] = sample_res["茎像素半径"] - sample_res["小维管束到茎中心的平均像素距离"]
        sample_res["内环像素面积"] = math.pi * sample_res["小维管束到茎中心的平均像素距离"] ** 2
        sample_res["皮像素面积"] = sample_res["茎像素面积"] - sample_res["内环像素面积"]
        sample_res["小维管束面积密度（%）"] = 100 * sample_res["小维管束像素面积之和"] / sample_res["皮像素面积"]
        sample_res["大维管束面积密度（%）"] = 100 * sample_res["大维管束像素面积之和"] / (sample_res["茎像素面积_减空腔后"] - sample_res["皮像素面积"])
        sample_res["空腔面积/茎面积（未减空腔）（%）"] = 100 * sample_res["空腔像素面积"] / sample_res["茎像素面积"]        
            
    all_res[sample_name] = sample_res        

0620删掉多余比例尺/L47
比例尺像素长度： 69
stem_center: (800, 812)
*****
0620删掉多余比例尺/L50
比例尺像素长度： 68
stem_center: (876, 808)
*****
hole_area_pixel: 79403.0
hole_area_pixel: 1197.0
0620删掉多余比例尺/F11
比例尺像素长度： 70
stem_center: (910, 841)
*****
0620删掉多余比例尺/F12
比例尺像素长度： 69
stem_center: (1024, 1025)
*****


In [10]:
# 查看直接导出的数据
all_res_df = pd.DataFrame(all_res).T
all_res_df.head(2)

,比例尺像素长度,空腔像素面积集,空腔像素面积,茎中心,茎像素半径,茎像素直径,茎像素周长,茎像素面积,茎像素面积_减空腔后,大维管束像素面积集,...,大维管束面积之和/茎面积_减空腔后（%）,小维管束面积之和/茎面积_减空腔后（%）,大维管束到茎中心的平均距离/茎半径（%）,小维管束到茎中心的平均距离/茎半径（%）,皮像素厚度,内环像素面积,皮像素面积,小维管束面积密度（%）,大维管束面积密度（%）,空腔面积/茎面积（未减空腔）（%）
0620删掉多余比例尺/L47,69,[],0.0,"(800, 812)",802,1604,4948.881554,1918178.0,1918178.0,"[516.0, 535.5, 359.5, 410.5, 489.5, 479.5, 567...",...,8.057933,2.02867,72.373114,95.048334,39.712359,1825524.350389,92653.649611,41.998885,8.46691,0.0
0620删掉多余比例尺/L50,68,"[79403.0, 1197.0]",80600.0,"(876, 808)",835,1670,5109.40781,2060734.0,1980134.0,"[598.5, 394.5, 967.0, 578.5, 583.0, 523.5, 529...",...,7.147829,2.82749,73.640784,95.029191,41.506254,1978048.285396,82685.714604,67.711938,7.459313,3.911228


In [11]:
# 存储直接导出的数据
savepath = wd + "维管束切片图像/"+"0620删掉多余比例尺/res20220713.xlsx"
sheet_name = "直接导出数据"
write2excel(all_res_df,savepath,sheet_name)

## 2.2 加工直接导出的数据，形成一级加二级数据

In [12]:
# 提取像素相关数据，进行比例转换
all_res_cols = all_res_df.columns # 直接导出数据的所有变量
pixel_cols = all_res_cols[all_res_cols.str.contains("像素")]
pixel_cols = pixel_cols[~pixel_cols.str.contains("集")].tolist() # 仅与像素有关的变量

remain_cols = pd.Series([x for x in all_res_cols if x not in pixel_cols])
remain_cols = remain_cols[~remain_cols.str.contains("集")].tolist() # 除去像素有关变量后剩余的变量

In [16]:
pixel_df = all_res_df[pixel_cols] # 提取与像素有关的数据
remain_df = all_res_df[remain_cols] # 提取剩余数据
pixel_df["缩放比例"] = 500 / pixel_df["比例尺像素长度"] # 计算缩放比例，500 微米除以比例尺的像素长度，得到以微米/像素为单位的缩放比例
pixel_df.head(2)

/var/folders/6w/tg5lm1lj5yd_wm8m4vf9wplc0000gn/T/ipykernel_89837/1486886299.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pixel_df["缩放比例"] = 500 / pixel_df["比例尺像素长度"] # 计算缩放比例，500 微米除以比例尺的像素长度，得到以微米/像素为单位的缩放比例


,比例尺像素长度,空腔像素面积,茎像素半径,茎像素直径,茎像素周长,茎像素面积,茎像素面积_减空腔后,大维管束像素面积之和,大维管束平均像素面积,大维管束到茎中心的平均像素距离,小维管束像素面积之和,小维管束平均像素面积,小维管束到茎中心的平均像素距离,总维管束像素面积之和,皮像素厚度,内环像素面积,皮像素面积,缩放比例
0620删掉多余比例尺/L47,69,0.0,802,1604,4948.881554,1918178.0,1918178.0,154565.5,508.439145,580.432371,38913.5,162.817992,762.287641,193479.0,39.712359,1825524.350389,92653.649611,7.246377
0620删掉多余比例尺/L50,68,80600.0,835,1670,5109.40781,2060734.0,1980134.0,141536.6,449.32254,614.900549,55988.1,245.561842,793.493746,197524.7,41.506254,1978048.285396,82685.714604,7.352941


In [17]:
# 进行比例转换，对长度变量，乘以缩放比例，对面积变量，乘以缩放比例的平方
physics_df = pd.DataFrame()
for col in pixel_cols[1:]:
    col_name = col.replace('像素', '')
    print(col_name)
    if "面积" in col_name:            
        physics_df[col_name] = pixel_df[col] * (pixel_df["缩放比例"] ** 2)
    else:
        physics_df[col_name] = pixel_df[col] * pixel_df["缩放比例"]       

空腔面积
茎半径
茎直径
茎周长
茎面积
茎面积_减空腔后
大维管束面积之和
大维管束平均面积
大维管束到茎中心的平均距离
小维管束面积之和
小维管束平均面积
小维管束到茎中心的平均距离
总维管束面积之和
皮厚度
内环面积
皮面积


In [18]:
# 将经比例调整后的数据与剩余数据合并
output = pd.concat([physics_df,remain_df],axis=1)
output["小维管束数量密度"] = output["小维管束数量"] / output["茎周长"] # 计算小维管束数量密度
output.head(2)

,空腔面积,茎半径,茎直径,茎周长,茎面积,茎面积_减空腔后,大维管束面积之和,大维管束平均面积,大维管束到茎中心的平均距离,小维管束面积之和,...,大维管束数量/总维管束数量（%）,大维管束面积之和/总维管束面积之和（%）,大维管束面积之和/茎面积_减空腔后（%）,小维管束面积之和/茎面积_减空腔后（%）,大维管束到茎中心的平均距离/茎半径（%）,小维管束到茎中心的平均距离/茎半径（%）,小维管束面积密度（%）,大维管束面积密度（%）,空腔面积/茎面积（未减空腔）（%）,小维管束数量密度
0620删掉多余比例尺/L47,0.0,5811.594203,11623.188406,35861.460534,100723482.461668,100723482.461668,8116230.833858,26698.127743,4206.031671,2043346.985927,...,55.985267,79.887481,8.057933,2.02867,72.373114,95.048334,41.998885,8.46691,0.0,0.006665
0620删掉多余比例尺/L50,4357698.961938,6139.705882,12279.411765,37569.175075,111415116.782007,107057417.820069,7652281.574394,24292.957379,4521.327569,3027038.278547,...,58.01105,71.65514,7.147829,2.82749,73.640784,95.029191,67.711938,7.459313,3.911228,0.006069


In [20]:
traits = pd.read_excel(savepath,sheet_name="性状表") # 读入性状表
cols_order = traits["性状"].tolist() # 按照性状表的顺序设置输出数据的顺序
output = output[cols_order]
output.head(2)

,空腔面积,茎半径,茎直径,茎周长,茎面积,茎面积_减空腔后,大维管束数量,大维管束面积之和,大维管束平均面积,大维管束到茎中心的平均距离,...,小维管束面积之和/茎面积_减空腔后（%）,大维管束到茎中心的平均距离/茎半径（%）,小维管束到茎中心的平均距离/茎半径（%）,皮厚度,内环面积,皮面积,小维管束数量密度,小维管束面积密度（%）,大维管束面积密度（%）,空腔面积/茎面积（未减空腔）（%）
0620删掉多余比例尺/L47,0.0,5811.594203,11623.188406,35861.460534,100723482.461668,100723482.461668,304,8116230.833858,26698.127743,4206.031671,...,2.02867,72.373114,95.048334,287.770715,95858241.461317,4865241.000351,0.006665,41.998885,8.46691,0.0
0620删掉多余比例尺/L50,4357698.961938,6139.705882,12279.411765,37569.175075,111415116.782007,107057417.820069,315,7652281.574394,24292.957379,4521.327569,...,2.82749,73.640784,95.029191,305.193046,106944652.108348,4470464.673659,0.006069,67.711938,7.459313,3.911228


In [21]:
# 保存一级加二级数据
savepath = wd + "维管束切片图像/"+"0620删掉多余比例尺/res20220713.xlsx"
sheet_name = "一级加二级数据"
write2excel(output,savepath,sheet_name)